# REINFORCE and Actor-Critic

## 0. Setups

Import required libraries

In [2]:
import numpy as np
import gym
import time
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from sklearn.linear_model import LinearRegression, Lasso

# For Colab users, turn this into true
colab = True

Select hardware to use - GPU or CPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)

cuda


For rendering **[COLAB USE ONLY!]**

In [4]:
if colab:
    !pip install swig
    !pip install gym pyvirtualdisplay > /dev/null 2>&1
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !apt-get update > /dev/null 2>&1
    !apt-get install cmake > /dev/null 2>&1
    !pip install --upgrade setuptools 2>&1
    !pip install ez_setup > /dev/null 2>&1
    !pip3 install box2d-py
    !pip3 install gym[Box_2D]
    !pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp310-cp310-linux_x86_64.whl size=2812077 sha256=fa64a1ca26c8aeb00b6493ebac1c29a9f9d577bd83513da1c1efed39cd7c855b
  Stored in directory: /root/.cache/pip/wheels/47/01/d2/6a780da77ccb98b1d2facdd520a8d10838a03b590f6f8d50c0
Successfully built box2d-py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Build Environment and check MDP size

In [5]:
env = gym.make('CartPole-v1')
env.seed(500)
torch.manual_seed(500)

# Configure MDP
gamma = 0.99
state_dim = env.observation_space.low.size
num_action = env.action_space.n
print('Dimension of state space / number of actions : %d / %d'%(state_dim, num_action))

Dimension of state space / number of actions : 4 / 2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

## 1. Create an policy instance

 Define policy network

In [6]:
class Policy(nn.Module):
    def __init__(self, state_dim, num_action, hidden_size1, hidden_size2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # TODO_1 : Define fc3 layer, of which output dim is num_action.
        # self.fc3 =
        self.fc3 = nn.Linear(hidden_size2, num_action)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        action_score = self.fc3(x)

        # TODO_2 : Return the probability to choose each actions with F.softmax().
        # return
        return F.softmax(action_score, dim=1)


## 2. REINFORCE

```python
m = Categorial(probs)
```
makes neural network output computation graph (gradient) into discrete probability distribution, thus it is possible to calculate $\nabla_\theta\log{\pi_\theta(a|s)}$

In [7]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    state = state.to(device)
    probs = pi(state)

    m = Categorical(probs)
    action = m.sample()

    return action.item(), m.log_prob(action)

def sample_trajectory(data, T):
    # Reset environment to get new trajectory.
    state = env.reset()
    r_sum, r_sum_discount = 0, 0

    for t in range(T):
        # TODO_3 : Get an action from current policy and rollout.
        # action, log_prob =
        action, log_prob = select_action(state)
        next_state, reward, done, _ = env.step(action)
        r_sum += reward
        r_sum_discount += reward * (gamma ** t)

        # Store the data.
        data['log_pi'].append(-log_prob) # (-) sign for gradient ascent
        data['state'].append(state)
        data['next_state'].append(next_state)
        data['reward'].append(reward)

        # Step
        state = next_state
        if done:
            break

    return r_sum, r_sum_discount

REINFORCE algorithm approximate gradient for policy parameter $\theta$ with sampled trajectory
$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum^N_{i=1} \big( \sum^T_{t=0}\nabla_\theta\log{\pi_\theta}(a_t|s_t) \big) \big( \sum^T_{t=0} \gamma^t r(s_t,a_t) \big)$$


With further approximation and use of baseline,
$$ \nabla_\theta J(\theta) \approx \frac{1}{N} \sum^N_{i=1} \sum^T_{t=0} \big( \nabla_\theta\log{\pi_\theta(a_t|s_t)} \big) \big( Q(s_t,a_t) - v(s_t) \big)$$


For REINFORCE, we use
$$Q(s_t, a_t) \approx \sum^T_{t'=t} \gamma^t r(s_{t'}, a_{t'})$$

$$v^{\pi_\theta}(s_0) \approx \sum^N_{i=0} \sum^T_{t=0} \gamma^t r^i(s_t, a_t)$$

(Note that we use universial baseline!)

In [9]:
def calculate_PG(pi_returns_discounted, dataset):
    pi_loss = 0
    for data in dataset:
        advantage, DCR = [], 0
        for r in reversed(data['reward']):
            # TODO_4 : Caculate discounted return from t=i.
            # Hint : reversed() will give saved rewards in reversed order.
            # r_T, r_{T-1}, ... r_0
            # DCR(T) = r_T
            # DCR(T-1) = r_{T-1} + gamma * r_T
            # DCR =
            DCR = r + gamma * DCR

            # Q(s,a) is replaced with discounted sum of rewards (DCR)
            # v(s) is replaced with empirical v(s_0)
            advantage.insert(0, DCR - np.mean(pi_returns_discounted))

        # TODO_optional : alternate between two losses to see difference!
        # pi_loss_vanilla = [log_pi * DCR for log_pi in data['log_pi']]
        pi_loss_baseline = [log_pi * a for log_pi, a in zip(data['log_pi'], advantage)]

        # Take mean value
        # pi_loss += torch.cat(pi_loss_vanilla).sum()
        pi_loss += torch.cat(pi_loss_baseline).sum()
    return pi_loss / num_trajs

In [10]:
num_epochs = 100 # Num. of gradient steps
num_trajs = 100 # N
T = 10000 # T
log_interval = 5
total_time = []

pi = Policy(state_dim, num_action, 128, 128).to(device)
optimizer_pi = optim.Adam(pi.parameters(), lr=1e-3)

# For logging
pi_returns, pi_returns_discounted = [], []

for epoch in range(num_epochs):
    start_epoch = time.time()

    # On-policy dataset
    # We cannot reuse samples!
    dataset = []

    # Collect trajectories to perform gradient step.
    for N in range(num_trajs):
        data = {'log_pi':[], 'state':[], 'next_state':[], 'reward':[]}
        r_sum, r_sum_discount = sample_trajectory(data, T)
        dataset.append(data)

        # For logging - store most recent N trajectories
        pi_returns.append(r_sum)
        pi_returns_discounted.append(r_sum_discount)
        if len(pi_returns) > num_trajs:
            pi_returns.pop(0)
            pi_returns_discounted.pop(0)

    # Perform pocliy gradient step
    optimizer_pi.zero_grad()
    pi_loss = calculate_PG(pi_returns_discounted, dataset)
    pi_loss.backward()
    optimizer_pi.step()

    # Logging - print most recent epoch result
    epoch_time = time.time() - start_epoch
    total_time.append(epoch_time)
    if epoch % log_interval == 0:
        time_elapsed = np.sum(total_time)
        time_remain = np.mean(total_time) * num_epochs - time_elapsed
        print('Epoch {}\tReturn_mean: {:.2f}\tReturn_std: {:.2f}\tTime(Elapsed/Remain): {:.2f}/{:.2f} (mins)'.format(
            epoch, np.mean(pi_returns), np.std(pi_returns), time_elapsed/60, time_remain/60))

Epoch 0	Return_mean: 25.07	Return_std: 12.56	Time(Elapsed/Remain): 0.12/11.94 (mins)
Epoch 5	Return_mean: 27.69	Return_std: 12.68	Time(Elapsed/Remain): 0.53/8.28 (mins)
Epoch 10	Return_mean: 37.30	Return_std: 16.57	Time(Elapsed/Remain): 1.03/8.33 (mins)
Epoch 15	Return_mean: 45.94	Return_std: 21.99	Time(Elapsed/Remain): 1.64/8.62 (mins)
Epoch 20	Return_mean: 51.04	Return_std: 24.29	Time(Elapsed/Remain): 2.36/8.87 (mins)
Epoch 25	Return_mean: 61.74	Return_std: 32.33	Time(Elapsed/Remain): 3.21/9.15 (mins)
Epoch 30	Return_mean: 72.95	Return_std: 37.89	Time(Elapsed/Remain): 4.17/9.27 (mins)
Epoch 35	Return_mean: 84.15	Return_std: 37.02	Time(Elapsed/Remain): 5.30/9.42 (mins)
Epoch 40	Return_mean: 88.79	Return_std: 35.70	Time(Elapsed/Remain): 6.54/9.42 (mins)
Epoch 45	Return_mean: 103.56	Return_std: 55.06	Time(Elapsed/Remain): 8.03/9.42 (mins)
Epoch 50	Return_mean: 164.24	Return_std: 59.67	Time(Elapsed/Remain): 10.04/9.65 (mins)
Epoch 55	Return_mean: 204.02	Return_std: 87.91	Time(Elapsed/Rem

## 3. Actor-Critic (Linear Architecture)

In [ ]:
from numpy import linalg as LA

# Calculate feature vector
# state[0] : Cart pos
# state[1] : Cart speed
# state[2] : Pole angle
# state[3] : Pole velocity at tip

state2 = [-0.12, 0, 0.12] # termination condition
state3 = [-1, 0, 1]

mu = []
for s2 in state2:
    for s3 in state3:
        mu.append([s2, s3])

def state2feature(state):
    phi = []
    for f in mu:
        rad_base = LA.norm(np.array(state[-2:])-np.array(f)) ** 2
        phi.append(np.exp(-0.5*rad_base))

    return np.array(phi)


def calculate_vf(dataset, vf):
    X, y = [], []

    for data in dataset:
        for s, next_s, r in zip(data['state'], data['next_state'], data['reward']):
            v = state2feature(s)
            Q = r
            if vf is not None:
                Q = r + gamma * vf.predict(state2feature(next_s).reshape(1, -1))[0]
            X.append(v)
            y.append(Q)

    return X, y


def get_advantage(data, vf):
    advantage, baseline = [], []

    for s, next_s, r in zip(data['state'], data['next_state'], data['reward']):
        v = vf.predict(state2feature(s).reshape(1, -1))[0]
        v_next = vf.predict(state2feature(next_s).reshape(1, -1))[0]
        # TODO_5 : Complete advantage calculation by calculating Q-value.
        # Hint: SARSA style!
        # Q =
        # A =
        Q = r + gamma * v_next
        A = Q - v

        advantage.append(A)
        baseline.append(v)

    return advantage, baseline


def calculate_AC_PG(vf, pi_returns_discounted, dataset):
    pi_loss = 0
    for data in dataset:
        # For linear Actor-Critic
        advantage = []
        _, v = get_advantage(data, vf)
        DCR = 0
        for i, r in enumerate(reversed(data['reward'])):
            # TODO_6 : Compute DCR and advantage with baseline.
            # DCR =
            # advantage.insert(0, """blank""") # For practical algorithm, we just adopt baseline.
            DCR = r + gamma * DCR
            advantage.insert(0, DCR - v[i])

        # Compute each element of gradient
        pi_loss_linear_vf = [log_pi * a for log_pi, a in zip(data['log_pi'], advantage)]

        # Sums up log_prob * weight
        pi_loss += torch.cat(pi_loss_linear_vf).sum()

    return pi_loss / num_trajs

In [ ]:
num_epochs = 100
num_trajs = 100
T = 10000
log_interval = 5
total_time = []

pi = Policy(state_dim, num_action, 128, 128).to(device)
optimizer_pi = optim.Adam(pi.parameters(), lr=1e-3)
vf = None

# For logging
pi_returns, pi_returns_discounted = [], []

dataset_vf = []
for epoch in range(num_epochs):
    start_epoch = time.time()

    # On-policy dataset
    dataset = []

    # Collect trajectories to perform gradient step
    for N in range(num_trajs):
        data = {'log_pi':[], 'state':[], 'next_state':[], 'reward':[]}
        r_sum, r_sum_discount = sample_trajectory(data, T)
        dataset.append(data)
        dataset_vf.append(data)

        # For logging - store most recent N trajectories
        pi_returns.append(r_sum)
        pi_returns_discounted.append(r_sum_discount)
        if len(pi_returns) > num_trajs:
            pi_returns.pop(0)
            pi_returns_discounted.pop(0)

    ### NEW : update critic ###
    X, y = calculate_vf(dataset_vf, vf)
    vf = LinearRegression().fit(X, y)

    # Perform pocliy gradient step
    optimizer_pi.zero_grad()
    pi_loss = calculate_AC_PG(vf, pi_returns_discounted, dataset)
    pi_loss.backward()
    optimizer_pi.step()

    # Logging - print most recent epoch result
    epoch_time = time.time() - start_epoch
    total_time.append(epoch_time)
    if epoch % log_interval == 0:
        dataset_vf = []
        time_elapsed = np.sum(total_time)
        time_remain = np.mean(total_time) * num_epochs - time_elapsed
        print('Epoch {}\tReturn_mean: {:.2f}\tReturn_std: {:.2f}\tTime(Elapsed/Remain): {:.2f}/{:.2f} (mins)'.format(
            epoch, np.mean(pi_returns), np.std(pi_returns), time_elapsed/60, time_remain/60))

Epoch 0	Return_mean: 21.15	Return_std: 9.83	Time(Elapsed/Remain): 0.16/15.54 (mins)
Epoch 5	Return_mean: 29.13	Return_std: 16.91	Time(Elapsed/Remain): 1.01/15.86 (mins)
Epoch 10	Return_mean: 40.84	Return_std: 22.12	Time(Elapsed/Remain): 2.15/17.43 (mins)
Epoch 15	Return_mean: 50.96	Return_std: 25.50	Time(Elapsed/Remain): 3.38/17.75 (mins)
Epoch 20	Return_mean: 64.21	Return_std: 35.60	Time(Elapsed/Remain): 5.06/19.04 (mins)
Epoch 25	Return_mean: 92.70	Return_std: 45.66	Time(Elapsed/Remain): 7.26/20.66 (mins)
Epoch 30	Return_mean: 141.32	Return_std: 67.92	Time(Elapsed/Remain): 10.66/23.72 (mins)
Epoch 35	Return_mean: 210.06	Return_std: 87.66	Time(Elapsed/Remain): 15.78/28.06 (mins)
Epoch 40	Return_mean: 267.13	Return_std: 100.51	Time(Elapsed/Remain): 22.92/32.98 (mins)
Epoch 45	Return_mean: 354.81	Return_std: 113.78	Time(Elapsed/Remain): 32.17/37.76 (mins)
Epoch 50	Return_mean: 420.92	Return_std: 102.74	Time(Elapsed/Remain): 42.96/41.28 (mins)
Epoch 55	Return_mean: 442.54	Return_std: 85.

## 4. Visualize result

For rendering **[COLAB USE ONLY!]**

In [ ]:
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def show_video(env_id):
    mp4list = glob.glob('video/'+env_id+'/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env, env_id):
#    env = Monitor(env, './video', force=True)
    env=RecordVideo(env, './video/'+env_id,  episode_trigger = lambda episode_number: True)
    return env

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
if colab:
  env = wrap_env(env, env_id)
state = env.reset()

while True:
    env.render()
    action, log_prob = select_action(state)
    state, reward, done, info = env.step(action)
    if done:
        break

env.close()
if colab:
    show_video(env_id)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym